from: https://github.com/SadmanSakib93/Stratified-k-fold-cross-validation-Image-classification-keras/blob/master/stratified_K_fold_CV.ipynb

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = 1000000000
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2023-05-10 11:24:15.655542: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 11:24:19.549425: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
"""from google.colab import drive
drive.mount('/content/drive')
!ls
import sys
# This is the path to where in google drive the code is stored!
root_path = '/content/drive/My Drive/'
sys.path.append(root_path)
"""

root_path = '/workspaces/proposal-ta/images/'

In [3]:
### UBAH NAMA FILE MODEL SESUAI PENGUJIAN DI SINI
dataset_folder_name = os.path.join(root_path, '')
source_files = []
class_labels = ['a', 'ba', 'ca', 'da', 'ga', 
                'ja', 'ka','la','ma', 'na', 
                'nga', 'nya','pa', 'ra', 'sa', 
                'ta', 'wa', 'ya']
X = []
Y = []

img_rows, img_cols = 120, 120 # input image dimensions
train_path = os.path.join(dataset_folder_name, 'train')
validation_path = os.path.join(dataset_folder_name, 'validation')
test_path = os.path.join(dataset_folder_name, 'test')

In [4]:
# augmentasi data
# bisa jadi sebelum atau setelah split

In [5]:
def transfer_between_folders(source, dest, split_rate):
    """ Based on the split ratio this function moves some portion of the source folder to destination folder!

        Args:
            source: str
                Source folder's path
            dest: str
                Destination folder's path
            split_rate: float
                Ratio of files to move from source to dest location

    """
    global source_files
    source_files = os.listdir(source)
    if(len(source_files) != 0):
        transfer_file_numbers = int(len(source_files)*split_rate)
        transfer_index = random.sample(
            range(0, len(source_files)), transfer_file_numbers)
        for each_index in transfer_index:
            shutil.move(os.path.join(source, str(source_files[each_index])), os.path.join(
                dest, str(source_files[each_index])))

    else:
        print("No file moved. Source empty!")


def transfer_all_class_between_folders(source, dest, split_rate):
    """ Transfer the files from source to dest for all the classes. This function calls the 'transfer_between_folders' to actually perform the transfer.

        Args:
            source: str
                Source folder's path
            dest: str
                Destination folder's path
            split_rate: float
                Ratio of files to move from source to dest location

    """
    for label in class_labels:
        transfer_between_folders(os.path.join(dataset_folder_name, source, label),
                                 os.path.join(
                                     dataset_folder_name, dest, label),
                                 split_rate)


def my_metrics(y_true, y_pred):
    """ Calculate accuracy, precision, and f1 score of the model's prediction with respect to true labels.

        Args:
            y_true: list/array
                All true class labels
            y_pred: list/array
                All predicted class labels

        Returns:
            accuracy: float
                Accuracy measure of the model
            precision: float
                Precision measure of the model
            f1_Score: float
                F1-score measure of the model

    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    f1_Score = f1_score(y_true, y_pred, average='weighted')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1_Score))
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1_Score

In [6]:
transfer_all_class_between_folders('test','train', 1.0)
#transfer_all_class_between_folders('validation','train', 1.0)

In [7]:
transfer_all_class_between_folders('train', 'test', 0.20)

In [8]:
def prepare_name_with_labels(folder_name, dataset_type='train'):
    """ Prepare the file names (X) and the class labels (Y) from folder location of images.

        Args:
            folder_name: str
                Source folder's path

    """
    source_files = os.listdir(os.path.join(dataset_folder_name, dataset_type, folder_name))
    y_label = 0
    for i in range(len(class_labels)):
        if(folder_name == class_labels[i]):
            y_label = i
    for val in source_files:
        X.append(val)
        Y.append(y_label)

In [9]:
# Organize file names and class labels in X and Y variables
for i in range(len(class_labels)):
    prepare_name_with_labels(class_labels[i])

In [10]:
X = np.asarray(X)
Y = np.asarray(Y)

In [11]:
# arsitektur
batch_size = 32
epoch = 15
num_of_channels = 3
number_of_class_labels = len(class_labels)

In [12]:
"""
    Note that, this model structure is a very basic one. 
    To achieve better performance, you should change the model structure and hyperparameters according to your needs and data. 
    So, optimize the structure of the model!
"""

def get_model():
    activation_function = 'relu'

    model = Sequential()
    model.add(Conv2D(64, (3,3), padding='same',
                     activation=activation_function, input_shape=(img_rows, img_cols, num_of_channels)))
    model.add(Conv2D(64, (3,3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3,3), padding='same',
                     activation=activation_function))
    model.add(Conv2D(32, (3,3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(16, (3,3), padding='same',
                     activation=activation_function))
    model.add(Conv2D(16, (3,3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    #model.add(Conv2D(64, (3,3), padding='same',
    #                 activation=activation_function))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))
    
    #model.add(Conv2D(64, (3,3), padding='same',
    #                 activation=activation_function))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    #model.add(Conv2D(128, (3,3), padding='same',
    #                 activation=activation_function))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation=activation_function))
    #model.add(Dropout(0.1))
    #model.add(Dense(32, activation=activation_function))
    #model.add(Dropout(0.1))
    #model.add(Dense(16, activation=activation_function))
    #model.add(Dropout(0.1))
    model.add(Dense(number_of_class_labels, activation='softmax'))

    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


model = get_model()
model.summary()
#

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 120, 120, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 118, 118, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 59, 59, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 59, 59, 32)        18464     
                                                                 
 conv2d_3 (Conv2D)           (None, 57, 57, 32)        9248      
                                                        

In [13]:
#model = tf.keras.Sequential([
#    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(120, 120, 3)),
#    tf.keras.layers.MaxPooling2D(2, 2),
#    tf.keras.layers.Dropout(0.25),
#    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
#    tf.keras.layers.MaxPooling2D(2, 2),
#    tf.keras.layers.Dropout(0.25),
#    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
#    tf.keras.layers.MaxPooling2D(2, 2),
#    tf.keras.layers.Dropout(0.25),
#    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
#    tf.keras.layers.MaxPooling2D(2, 2),
#    tf.keras.layers.Dropout(0.25),
#    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
#    tf.keras.layers.MaxPooling2D(2, 2),
#    tf.keras.layers.Dropout(0.25),
#    #tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
#    #tf.keras.layers.MaxPooling2D(2, 2),
#    tf.keras.layers.Flatten(),
#    #tf.keras.layers.Dropout(0.2),
#    tf.keras.layers.Dense(128, activation='relu'),
#    tf.keras.layers.Dense(18, activation='softmax')
#])
#
#optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
#
#model.compile(
#    #optimizer='adam', 
#    optimizer=optimizer,
#    loss='categorical_crossentropy',
#    metrics=['accuracy'],
#)
#
#model.summary()

In [14]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=10, shuffle=True)
skf.get_n_splits(X, Y)
fold_num = 0
for train_index, val_index in skf.split(X, Y):
    # First cut all images from validation to train (if any exists)
    transfer_all_class_between_folders('validation', 'train'
                                       , 1.0)
    fold_num += 1
    print("Results for fold", fold_num)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for each_index in range(len(X_val)):
        class_label = ''
        for i in range(len(class_labels)):
            if(Y_val[each_index] == i):
                class_label = class_labels[i]
        # Then, copy the validation images to the validation folder
        shutil.move(os.path.join(dataset_folder_name, 'train', class_label, X_val[each_index]),
                    os.path.join(dataset_folder_name, 'validation', class_label, X_val[each_index]))

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.20,
        fill_mode="nearest",
        rotation_range=90,
        height_shift_range=0.1,
        width_shift_range=0.1,
        vertical_flip=True,
        horizontal_flip=True)
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)
    
    # fit model
    history = model.fit(train_generator,
                        epochs=epoch)

    predictions = model.predict(validation_generator, verbose=1)
    y_predictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    
    # evaluate validation performance
    print("***Performance on Validation data***")
    val_acc, val_prec, val_fScore = my_metrics(true_classes, y_predictions)

Results for fold 1
Found 1128 images belonging to 18 classes.
Found 116 images belonging to 18 classes.
Epoch 1/15


2023-05-10 11:24:25.897328: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


36/36 [==============================] - 57s 2s/step - loss: 2.8968 - accuracy: 0.0488
Epoch 2/15
36/36 [==============================] - 54s 2s/step - loss: 2.8908 - accuracy: 0.0612
Epoch 3/15
36/36 [==============================] - 52s 1s/step - loss: 2.8907 - accuracy: 0.0514
Epoch 4/15
36/36 [==============================] - 53s 1s/step - loss: 2.8909 - accuracy: 0.0479
Epoch 5/15
36/36 [==============================] - 52s 1s/step - loss: 2.8907 - accuracy: 0.0470
Epoch 6/15
36/36 [==============================] - 53s 1s/step - loss: 2.8909 - accuracy: 0.0514
Epoch 7/15
36/36 [==============================] - 53s 1s/step - loss: 2.8918 - accuracy: 0.0470
Epoch 8/15
36/36 [==============================] - 53s 1s/step - loss: 2.8909 - accuracy: 0.0567
Epoch 9/15
36/36 [==============================] - 52s 1s/step - loss: 2.8906 - accuracy: 0.0559
Epoch 10/15
36/36 [==============================] - 52s 1s/step - loss: 2.8907 - accuracy: 0.0479
Epoch 11/15
36/36 [===========

2023-05-10 11:38:10.288607: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


4/4 [==============================] - 1s 259ms/step
***Performance on Validation data***
Accuracy  : 0.05172413793103448
Precision : 0.00267538644470868
f1Score : 0.005087620124364047
[[0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0]]
Results for fold 2
Found 1128 images belonging to 18 classes.
Found 116 images belonging to 18 classes.
Epoch 1/

...

/home/vscode/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2023-05-10 13:31:54.989946: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


36/36 [==============================] - 55s 2s/step - loss: 2.8905 - accuracy: 0.0478
Epoch 2/15
36/36 [==============================] - 54s 2s/step - loss: 2.8905 - accuracy: 0.0549
Epoch 3/15
36/36 [==============================] - 57s 2s/step - loss: 2.8905 - accuracy: 0.0523
Epoch 4/15
36/36 [==============================] - 55s 2s/step - loss: 2.8904 - accuracy: 0.0469
Epoch 5/15
36/36 [==============================] - 55s 2s/step - loss: 2.8905 - accuracy: 0.0567
Epoch 6/15
36/36 [==============================] - 54s 1s/step - loss: 2.8905 - accuracy: 0.0567
Epoch 7/15
36/36 [==============================] - 54s 1s/step - loss: 2.8905 - accuracy: 0.0567
Epoch 8/15
36/36 [==============================] - 55s 2s/step - loss: 2.8905 - accuracy: 0.0567
Epoch 9/15
36/36 [==============================] - 55s 2s/step - loss: 2.8905 - accuracy: 0.0567
Epoch 10/15
36/36 [==============================] - 54s 1s/step - loss: 2.8905 - accuracy: 0.0567
Epoch 11/15
36/36 [===========

2023-05-10 13:46:01.455511: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


4/4 [==============================] - 1s 230ms/step
***Performance on Validation data***
Accuracy  : 0.05217391304347826
Precision : 0.0027221172022684308
f1Score : 0.0051742723679482575
[[0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0]]


/home/vscode/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False 
)
predictions = model.predict(test_generator, verbose=1) 
y_predictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

test_acc, test_prec, test_fScore = my_metrics(true_classes, y_predictions)

==============TEST RESULTS============
Found 285 images belonging to 18 classes.


2023-05-10 13:53:54.707021: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


9/9 [==============================] - 2s 270ms/step
Accuracy  : 0.05263157894736842
Precision : 0.002770083102493075
f1Score : 0.005263157894736842
[[ 0  0  0  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 15  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 15  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 15  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 16  0  0  0  0  0  0  0  0  0  0 

/home/vscode/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
from sklearn.metrics import classification_report
clr = classification_report(true_classes, y_predictions,
                            labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 
                            target_names=['a', 'ba','ca','da','ga','ja','ka','la','ma','na','nga','nya','pa','ra','sa','ta','wa','ya'])
print(clr)

              precision    recall  f1-score   support

           a       0.00      0.00      0.00        16
          ba       0.00      0.00      0.00        16
          ca       0.00      0.00      0.00        16
          da       0.00      0.00      0.00        16
          ga       0.00      0.00      0.00        15
          ja       0.00      0.00      0.00        16
          ka       0.05      1.00      0.10        15
          la       0.00      0.00      0.00        16
          ma       0.00      0.00      0.00        16
          na       0.00      0.00      0.00        16
         nga       0.00      0.00      0.00        16
         nya       0.00      0.00      0.00        16
          pa       0.00      0.00      0.00        15
          ra       0.00      0.00      0.00        16
          sa       0.00      0.00      0.00        16
          ta       0.00      0.00      0.00        16
          wa       0.00      0.00      0.00        16
          ya       0.00    

/home/vscode/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vscode/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vscode/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
#MODEL_FILENAME = root_path+"/modelpengujian04_3x3_arsitektur2.h5"
#model.save(MODEL_FILENAME)